In [1]:
!pip install -q streamlit pyngrok torch openai-whisper pydub anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s e

In [31]:
%%writefile app.py
import streamlit as st
import whisper
import torch
import tempfile
import anthropic
import pandas as pd
import re
import requests
import io
from requests.auth import HTTPBasicAuth

# === Embed your Claude API Key here ===
claude_api_key = ""

# UI Config
st.set_page_config(page_title="📝 Meeting Whisperer", layout="centered")
st.title("Meeting Whisperer with Claude")

if not claude_api_key:
    st.warning("🔑 Claude API key not set.")
    st.stop()

client = anthropic.Anthropic(api_key=claude_api_key)

# Initialize session state
for key in ["transcript", "summary", "compliance", "task_matrix"]:
    if key not in st.session_state:
        st.session_state[key] = None

# Upload audio
uploaded_file = st.file_uploader("Upload your meeting audio", type=["mp3", "wav", "m4a"])

if uploaded_file and st.session_state.transcript is None:
    with tempfile.NamedTemporaryFile(delete=False) as tmp:
        tmp.write(uploaded_file.read())
        tmp_path = tmp.name

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = whisper.load_model("base").to(device)

    st.info("🔊 Transcribing audio...")
    result = model.transcribe(tmp_path, fp16=torch.cuda.is_available())
    st.session_state.transcript = result["text"]

transcript = st.session_state.transcript

if transcript:
    st.subheader("📄 Transcript")
    st.write(transcript)

    # === Claude Summary ===
    if st.session_state.summary is None:
        summary_prompt = f"""
You are a helpful assistant. Summarize the following meeting transcript and extract any action items or key decisions.

Transcript:
{transcript[:4000]}
"""
        with st.spinner("🤖 Summarizing with Claude..."):
            response = client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=800,
                messages=[{"role": "user", "content": summary_prompt}],
            )
            st.session_state.summary = response.content[0].text

    st.subheader("🧠 Summary & Action Items")
    st.write(st.session_state.summary)
    st.download_button("📄 Download Summary", st.session_state.summary, file_name="meeting_summary.txt")

    # === Task Matrix Extraction ===
    if st.session_state.task_matrix is None:
        task_prompt = f"""
Extract all actionable tasks discussed in this transcript and return a Markdown table in this format:

| Task | Assigned To | Deadline |

Use 'Unassigned' and 'Not specified' if needed.

Transcript:
{transcript[:4000]}
"""
        with st.spinner("📋 Extracting Task Matrix..."):
            task_resp = client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=800,
                messages=[{"role": "user", "content": task_prompt}],
            )
            st.session_state.task_matrix = task_resp.content[0].text

    st.subheader("✅ Task Matrix")
    st.code(st.session_state.task_matrix)

    # ✅ FIX: Parse Markdown Table (remove separator --- row)
    task_df = pd.DataFrame()
    table_match = re.search(r"(?P<table>(\|.+?\|[\r\n]+)+)", st.session_state.task_matrix)
    if table_match:
        table_text = table_match.group("table")

        # Remove separator row like | --- | --- | --- |
        lines = table_text.strip().splitlines()
        cleaned_lines = [line for line in lines if not re.match(r"^\|\s*-+\s*\|", line)]
        cleaned_table = "\n".join(cleaned_lines)

        try:
            task_df = pd.read_csv(io.StringIO(cleaned_table), sep="|", engine="python")
            task_df = task_df.dropna(axis=1, how="all")
            task_df.columns = [c.strip() for c in task_df.columns]
            st.dataframe(task_df)
        except Exception as e:
            st.warning(f"⚠️ Could not format task matrix: {e}")
    else:
        st.warning("⚠️ Could not extract Markdown table from Claude response.")

    # === Jira Integration ===
    def create_jira_ticket(base_url, project_key, email, api_token, summary, description_text):
        url = f"{base_url}/rest/api/3/issue"
        headers = {
            "Accept": "application/json",
            "Content-Type": "application/json"
        }
        auth = HTTPBasicAuth(email, api_token)

        description_doc = {
            "type": "doc",
            "version": 1,
            "content": [
                {
                    "type": "paragraph",
                    "content": [
                        {"type": "text", "text": description_text}
                    ]
                }
            ]
        }

        payload = {
            "fields": {
                "project": {"key": project_key},
                "summary": summary,
                "description": description_doc,
                "issuetype": {"name": "Task"}
            }
        }

        response = requests.post(url, headers=headers, auth=auth, json=payload)
        return response.status_code, response.json()

    with st.expander("🪄 Create Jira Tickets from Tasks"):
        jira_domain = st.text_input("Your Jira Domain (e.g. yourteam.atlassian.net)")
        jira_project = st.text_input("Jira Project Key (e.g. ENG)")
        jira_email = st.text_input("Your Jira Email", type="password")
        jira_token = st.text_input("Jira API Token", type="password")

        if st.button("🚀 Create Jira Tickets"):
            if not all([jira_domain, jira_project, jira_email, jira_token]):
                st.warning("Please fill in all Jira credentials.")
            elif task_df.empty:
                st.warning("No tasks available to send.")
            else:
                for _, row in task_df.iterrows():
                    title = row["Task"]
                    assignee = row["Assigned To"]
                    deadline = row["Deadline"]
                    description_text = (
                        f"Auto-generated from Meeting Whisperer\n\n"
                        f"Assigned To: {assignee}\nDeadline: {deadline}\n\n"
                        f"Transcript Summary:\n{st.session_state.summary}"
                    )

                    status, result = create_jira_ticket(
                        base_url=f"https://{jira_domain}",
                        project_key=jira_project,
                        email=jira_email,
                        api_token=jira_token,
                        summary=title,
                        description_text=description_text
                    )

                    if status == 201:
                        st.success(f"✅ Created: {title}")
                    else:
                        st.error(f"❌ Failed: {title} — {result}")

    # === Compliance & Sentiment ===
    if st.session_state.compliance is None:
        compliance_prompt = f"""
Analyze the following transcript for compliance and sentiment indicators. Return your response in three sections:

1. Sensitive Content: Identify any legally, financially, or ethically sensitive decisions or discussions.
2. Emotionally Charged Language: Highlight statements with strong negative or positive emotions.
3. Commitments Made: List any commitments made by name (e.g., "Alice will send the report by Friday").

Transcript:
{transcript[:4000]}
"""
        with st.spinner("🔍 Analyzing Compliance & Sentiment..."):
            compliance_response = client.messages.create(
                model="claude-3-haiku-20240307",
                max_tokens=800,
                messages=[{"role": "user", "content": compliance_prompt}],
            )
            st.session_state.compliance = compliance_response.content[0].text

    st.subheader("🧑‍⚖️ Compliance & Sentiment Tags")
    st.markdown(st.session_state.compliance)

    # === Ask a Question Section ===
    with st.expander("💬 Ask a Question about the Meeting"):
        user_question = st.text_input("Ask Claude anything about this meeting:")
        if st.button("Submit Question"):
            if user_question.strip():
                question_prompt = f"""
Here is a meeting transcript:
{transcript[:4000]}

Now answer the following question about the meeting:
{user_question}
"""
                with st.spinner("💬 Claude is answering..."):
                    answer = client.messages.create(
                        model="claude-3-haiku-20240307",
                        max_tokens=500,
                        messages=[{"role": "user", "content": question_prompt}],
                    )
                    st.success("✅ Answer:")
                    st.write(answer.content[0].text)
            else:
                st.warning("Please enter a question first.")

Overwriting app.py


In [32]:
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("")

# Start Streamlit in background
!streamlit run app.py &>/content/logs.txt &

# Now create the tunnel using HTTP
public_url = ngrok.connect("8501", "http")  # explicitly specify HTTP type
print("🌐 Streamlit Public URL:", public_url)


🌐 Streamlit Public URL: NgrokTunnel: "https://cdec-34-135-101-167.ngrok-free.app" -> "http://localhost:8501"
